# Health Monitor Demo: Норма vs Аномалия

Этот ноутбук демонстрирует полный цикл спектральной диагностики:
1. Генерация временного сигнала (норма и дефект)
2. Преобразование в спектр  
3. Сравнение с эталоном
4. Детекция аномалии

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from spectral_physics import (
    timeseries_to_spectrum,
    MaterialSignature,
    ChannelConfig,
    SpectralAnalyzer,
    HealthMonitor,
)

# Параметры
duration = 1.0  # seconds
dt = 0.001      # 1 ms
t = np.arange(0, duration, dt)

## 1. Генерация "нормального" сигнала

Симулируем вибрацию здорового подшипника:

In [ ]:
# Нормальный сигнал: основная частота 50 Гц + небольшая гармоника 120 Гц
signal_normal = (
    np.sin(2 * np.pi * 50 * t) +
    0.1 * np.sin(2 * np.pi * 120 * t) +
    0.05 * np.random.randn(len(t))  # Небольшой шум
)

plt.figure(figsize=(12, 3))
plt.plot(t[:500], signal_normal[:500])
plt.title("Нормальный сигнал (первые 0.5 сек)")
plt.xlabel("Время (сек)")
plt.ylabel("Амплитуда")
plt.grid(True)
plt.show()

## 2. Генерация "дефектного" сигнала

Добавляем частоту 200 Гц, характерную для дефекта:

In [ ]:
# Дефектный сигнал: добавляем сильную компоненту 200 Гц
signal_defect = (
    signal_normal +
    0.3 * np.sin(2 * np.pi * 200 * t)  # Дефект!
)

plt.figure(figsize=(12, 3))
plt.plot(t[:500], signal_defect[:500])
plt.title("Дефектный сигнал (первые 0.5 сек)")
plt.xlabel("Время (сек)")
plt.ylabel("Амплитуда")
plt.grid(True)
plt.show()

## 3. Преобразование в спектр

In [ ]:
# Преобразуем оба сигнала в спектры
spec_normal = timeseries_to_spectrum(signal_normal, dt=dt, window="hann")
spec_defect = timeseries_to_spectrum(signal_defect, dt=dt, window="hann")

# Конвертируем omega в Гц для удобства
freq_normal = spec_normal.omega / (2 * np.pi)
freq_defect = spec_defect.omega / (2 * np.pi)

# Визуализация спектров
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(freq_normal, spec_normal.power)
plt.title("Спектр нормального сигнала")
plt.xlabel("Частота (Гц)")
plt.ylabel("Мощность")
plt.xlim(0, 300)
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(freq_defect, spec_defect.power, color='red')
plt.title("Спектр дефектного сигнала")
plt.xlabel("Частота (Гц)")
plt.ylabel("Мощность")
plt.xlim(0, 300)
plt.grid(True)

plt.tight_layout()
plt.show()

## 4. Создание детектора и вычисление метрик

In [ ]:
# Создаем MaterialSignature на основе нормального спектра
signature = MaterialSignature(reference=spec_normal)

# Вычисляем расстояния
d_normal = signature.distance_l2(spec_normal)
d_defect = signature.distance_l2(spec_defect)

print(f"Расстояние (норма vs норма): {d_normal:.6f}")
print(f"Расстояние (норма vs дефект): {d_defect:.6f}")
print(f"\nУвеличение: {d_defect / (d_normal + 1e-10):.1f}x")

## 5. Детекция аномалии с порогом

In [ ]:
# Устанавливаем порог между нормой и дефектом
threshold = 0.1

is_normal_anomalous = signature.is_anomalous(spec_normal, threshold)
is_defect_anomalous = signature.is_anomalous(spec_defect, threshold)

print(f"Порог: {threshold:.3f}")
print(f"\nНормальный сигнал: {'❌ АНОМАЛИЯ' if is_normal_anomalous else '✅ OK'}")
print(f"Дефектный сигнал: {'❌ АНОМАЛИЯ' if is_defect_anomalous else '✅ OK'}")

## 6. Использование HealthMonitor (высокоуровневый API)

In [ ]:
# Создаем конфигурацию канала
config = ChannelConfig(
    name="bearing-vibration",
    dt=dt,
    window="hann",
    freq_min=10.0,   # Анализируем только 10-300 Гц
    freq_max=300.0
)

# Создаем анализатор
analyzer = SpectralAnalyzer(config)

# Анализируем сигналы
analyzed_normal = analyzer.analyze(signal_normal)
analyzed_defect = analyzer.analyze(signal_defect)

# Создаем монитор здоровья
monitor = HealthMonitor(reference=analyzed_normal, threshold=0.1)

# Проверяем статус
print("=" * 50)
print("РЕЗУЛЬТАТЫ МОНИТОРИНГА")
print("=" * 50)
print(f"Нормальный сигнал:")
print(f"  Score: {monitor.score(analyzed_normal):.4f}")
print(f"  Статус: {'ANOMALY' if monitor.is_anomalous(analyzed_normal) else 'OK'}")
print()
print(f"Дефектный сигнал:")
print(f"  Score: {monitor.score(analyzed_defect):.4f}")
print(f"  Статус: {'ANOMALY' if monitor.is_anomalous(analyzed_defect) else 'OK'}")
print("=" * 50)

## Выводы

✅ Система успешно различает нормальный и дефектный сигналы  
✅ Метрика L2-distance чувствительна к появлению новых частотных компонент  
✅ Высокоуровневый API (HealthMonitor) делает детекцию простой и понятной

**Практическое применение:** такой подход можно использовать для:
- Диагностики подшипников
- Мониторинга вибрации двигателей
- Детекции трещин в конструкциях
- Анализа электрических сигналов